# Part I

Need data to talk about data, and a model to talk about models...

### The Data

In [1]:
import pandas as pd

df = pd.read_csv('data/basketball.csv', parse_dates=[4])
df = df.sort_values(['name', 'date']).reset_index(drop=True)

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4986 entries, 0 to 4985
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   name      4986 non-null   object        
 1   position  4986 non-null   object        
 2   minutes   4986 non-null   int64         
 3   points    4986 non-null   int64         
 4   date      4986 non-null   datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 194.9+ KB


In [3]:
df.shape

(4986, 5)

In [4]:
df.sample(10)

,name,position,minutes,points,date
1818,JJ Redick,G,0,0,2020-08-13
2891,Landry Shamet,G,24,4,2020-09-09
4972,Zion Williamson,F,25,23,2020-08-03
1837,Ja Morant,G,41,22,2020-07-31
4905,Wesley Matthews,G,0,0,2020-08-13
3661,OG Anunoby,F,25,7,2020-08-10
698,D. Finney-Smith,F,30,8,2020-08-19
4920,Will Barton,F,0,0,2020-08-08
6,A. Cleveland,G,0,0,2020-08-11
539,Carsen Edwards,G,0,0,2020-09-11


In [5]:
df[df['name'] == 'Kawhi Leonard'].head(3)

,name,position,minutes,points,date
2593,Kawhi Leonard,F,33,28,2020-07-30
2594,Kawhi Leonard,F,27,24,2020-08-01
2595,Kawhi Leonard,F,37,27,2020-08-04


In [6]:
kawhi = df[df['name'] == 'Kawhi Leonard'].copy()

In [7]:
kawhi['points_1'] = kawhi['points'].shift(1)
kawhi['points_2'] = kawhi['points'].shift(2)

In [8]:
kawhi.head(10)

,name,position,minutes,points,date,points_1,points_2
2593,Kawhi Leonard,F,33,28,2020-07-30,NaN,NaN
2594,Kawhi Leonard,F,27,24,2020-08-01,28.0,NaN
2595,Kawhi Leonard,F,37,27,2020-08-04,24.0,28.0
2596,Kawhi Leonard,F,38,29,2020-08-06,27.0,24.0
2597,Kawhi Leonard,F,0,0,2020-08-08,29.0,27.0
2598,Kawhi Leonard,F,37,39,2020-08-09,0.0,29.0
2599,Kawhi Leonard,F,33,26,2020-08-12,39.0,0.0
2600,Kawhi Leonard,F,0,0,2020-08-14,26.0,39.0
2601,Kawhi Leonard,F,38,29,2020-08-17,0.0,26.0
2602,Kawhi Leonard,F,41,35,2020-08-19,29.0,0.0


In [9]:
df['points_1'] = df.groupby('name')['points'].shift(1)
df['points_2'] = df.groupby('name')['points'].shift(2)

In [10]:
df = df.dropna(subset=["points_1", "points_2"])

### The Objective

<font color="red">0 to 💯 real quick</font>

Predict \*points\* next game based on the points score for the last two games

In [11]:
target = 'points'
y = df[target]
X = df[['position', 'points_1', 'points_2']]

In [12]:
from sklearn.model_selection import train_test_split

`train_test_split` on time series data is a little different...

In [13]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.10, 
    random_state=42, 
    shuffle=False
)

### The Model

In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn.impute import SimpleImputer
# new
from sklearn_pandas import DataFrameMapper

In [15]:
mapper = DataFrameMapper([
    (['position'], [SimpleImputer(strategy="most_frequent"), LabelBinarizer()]),
    (['points_1'], [SimpleImputer(), StandardScaler()]), 
    (['points_2'], [SimpleImputer(), StandardScaler()]),
], df_out=True)

In [16]:
Z_train = mapper.fit_transform(X_train)
Z_test = mapper.transform(X_test)

In [17]:
model = LinearRegression()
model.fit(Z_train, y_train)

LinearRegression()

In [18]:
model.score(Z_train, y_train)

0.4381175473143679

In [19]:
from sklearn.metrics import mean_squared_error

In [20]:
mean_squared_error(y_test, model.predict(Z_test)) ** (1/2)

4.929029148411264

### DataFrameMapper (How + Why)

https://github.com/scikit-learn-contrib/sklearn-pandas

```pip install sklearn-pandas```

In [21]:
demo = pd.DataFrame({
    'position': ['C', 'SF', 'SG', 'PG']
})

pd.get_dummies(demo)

,position_C,position_PG,position_SF,position_SG
0,1,0,0,0
1,0,0,1,0
2,0,0,0,1
3,0,1,0,0


In [22]:
demo_2 = pd.DataFrame({
    'position': ['C', 'SF-SG', 'SG', 'C']
})

pd.get_dummies(demo_2)

,position_C,position_SF-SG,position_SG
0,1,0,0
1,0,1,0
2,0,0,1
3,1,0,0


In [23]:
new = pd.DataFrame({
    'position': ['🍔']
})

pd.get_dummies(new)

,position_🍔
0,1


### What to do instead...

In [24]:
X_train.sample(5)

,position,points_1,points_2
498,G,0.0,34.0
2573,G-F,0.0,0.0
32,F,0.0,4.0
3119,F,9.0,12.0
2155,F-G,29.0,18.0


In [25]:
lb = LabelBinarizer()
lb.fit(X_train['position'])
lb.transform(X_train['position'])

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [26]:
lb.classes_

array(['C', 'C-F', 'F', 'F-C', 'F-G', 'G', 'G-F'], dtype='<U3')

In [27]:
new = pd.DataFrame({
    'position': ['🍔']
})

In [28]:
lb.transform(new['position'])

array([[0, 0, 0, 0, 0, 0, 0]])

In [29]:
new = pd.DataFrame({
    'position': [None]
})

# lb.transform(new['position'])

In [30]:
mapper = DataFrameMapper([
    (['position'], [SimpleImputer(strategy="most_frequent"), LabelBinarizer()]),
    (['points_1'], [SimpleImputer(), StandardScaler()]), 
    (['points_2'], [SimpleImputer(), StandardScaler()]),
], df_out=True)

In [31]:
mapper.fit(X_train)
mapper.transform(X_train)[:10]

,position_C,position_C-F,position_F,position_F-C,position_F-G,position_G,position_G-F,points_1,points_2
2,0,0,0,0,0,1,0,-0.592296,-0.814399
3,0,0,0,0,0,1,0,-0.815025,-0.593534
4,0,0,0,0,0,1,0,-0.815025,-0.814399
5,0,0,0,0,0,1,0,-0.815025,-0.814399
6,0,0,0,0,0,1,0,-0.815025,-0.814399
7,0,0,0,0,0,1,0,-0.815025,-0.814399
8,0,0,0,0,0,1,0,-0.369567,-0.814399
9,0,0,0,0,0,1,0,-0.815025,-0.372668
10,0,0,0,0,0,1,0,-0.815025,-0.814399
11,0,0,0,0,0,1,0,-0.815025,-0.814399


But, maybe the best part about `mapper` is that you can put it in a pipeline...

In [32]:
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(mapper, model)
pipe.fit(X_train, y_train)

Pipeline(steps=[('dataframemapper',
                 DataFrameMapper(df_out=True,
                                 features=[(['position'],
                                            [SimpleImputer(strategy='most_frequent'),
                                             LabelBinarizer()]),
                                           (['points_1'],
                                            [SimpleImputer(),
                                             StandardScaler()]),
                                           (['points_2'],
                                            [SimpleImputer(),
                                             StandardScaler()])])),
                ('linearregression', LinearRegression())])

### The Pickle 🥒

In [33]:
import pickle

with open('pickles/pipe.pkl', 'wb') as f:
    pickle.dump(pipe, f)

In [34]:
del pipe

In [35]:
with open('pickles/pipe.pkl', 'rb') as f:
    pipe = pickle.load(f)

In [36]:
pipe.score(X_train, y_train)

0.4381175473143679

In [37]:
pipe.predict(X_train)[:10]

array([3.02392578, 2.98291016, 2.27246094, 2.27246094, 2.27246094,
       2.27246094, 3.77587891, 3.69384766, 2.27246094, 2.27246094])

In [38]:
X_train.sample(1).to_dict(orient='list')

{'position': ['F'], 'points_1': [27.0], 'points_2': [0.0]}

In [39]:
new = pd.DataFrame({
    'position': ['F'], 
    'points_1': [9.0], 
    'points_2': [8.0]
})

In [40]:
pipe.predict(new)

array([7.95166016])